*  This program computes 
*    1) the 4 summary statistics
*    2) the geometric means
*    3) some functions of interest
*    4) a test of normality  
*    5) maximum drawdown, etc

In [ ]:
import pandas as pd                     # To load data, we use the package pandas
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm                      # We use this package to do estimation
%matplotlib inline

# Load the monthly return of SP500 from 01/1934 to 12/2011

df = pd.read_excel('SP500.xlsx')  
Re = df.loc[:,"Return"]

# Compute the mean and variance

mu = np.mean(Re)
sig = np.var(Re)          # The variance, i.e., the square of the standard deviation
std = np.sqrt(sig)        # The standard deviation
sigma = std

print('(Monthly) Mean,   Std  \n')
print('        {0:.4f}   {1:.4f}\n '.format(mu, std))    

amu = 12*mu
astd = np.sqrt(12)*std
print('(Annualized) Mean,   Std  \n')
print('        {0:.4f}   {1:.4f}\n '.format(amu, astd))   

In [ ]:
# Compute the Sharpe Ratio

df2 = pd.read_excel('Riskfree.xlsx') 
Rate = df2.loc[:,"rate"]

ER = Re - Rate/100          # the excess return, i.e., return minus riskfree rate
                              #  divided by 100 b/c the rate data is in percentage points
mu2 = np.mean(ER)
sig2 = np.var(ER)          # The variance, i.e., the square of the standard deviation
std2 = np.sqrt(sig)        # The standard deviation

Sharpe = mu2/std2

print('The monthly and annulaized Sharpe ratios \n')
print('          {0:.2f}   {1:.2f}  \n'.format(Sharpe, np.sqrt(12)*Sharpe)) 

In [ ]:
# Compute the skewness and kurtosis

skew=0;                  # initialize it be zero
kurt=0;

T = len(df)             # Get the length, # of obvs (the headers of the Excel doesn't count)

for i in range(T):
    skew=skew + pow(Re[i]-mu,3)          # sums the 3rd power terms successively
    kurt=kurt + pow(Re[i]-mu,4)

skew=( skew / pow(sigma,3) ) / T               # take the average
kurt=( kurt / pow(sigma,4) ) / T 

print('(Monthly) skew,   kurt \n')
print('        {0:.4f}   {1:.4f}\n '.format(skew, kurt))    

In [ ]:
# Compute the 95% confidence for the skewness and kurtosis
#   Note first the 95% interval of the normal is [-1.96, 1.96],
#     which can be computed by:   x=norminv([0.025 0.975],0,1)
    
skewA=-1.96*np.sqrt(6/T)
skewB=1.96*np.sqrt(6/T)
 
kurtA=-1.96*np.sqrt(24/T)+3
kurtB=1.96*np.sqrt(24/T)+3

print(' Confidence interval for skew  \n')
print('        {0:.4f}   {1:.4f}\n '.format(skewA, skewB))
print(' Confidence interval for kurt \n')
print('        {0:.4f}   {1:.4f}\n '.format(kurtA, kurtB))    

In [ ]:
# Compute the wealth (accu return) and geometric mean 

Value = 1

for i in range(T):
    Value = Value * (1 + Re[i])

gmu=pow(Value, 1/T) - 1 


print('The accumulative returns \n')
print('          {0:.2f}\n'.format(Value))   

print('The sample mean and geometric mean  \n')
print('        {0:.4f}   {1:.4f}\n '.format(mu, gmu))    

In [ ]:
# Q1:  Percentage of up returns?
     
Up = 0

for j in range(T): 
    if Re[j]>0:
        Up = Up+1

UpPer = Up/T

print('The percentage of up months \n')
print('          {0:.2f} \n'.format(UpPer)) 

In [ ]:
# Q2: What is the accumulative return if we miss 5% of the best returns ?
# Assuming earning the average riskfree rate of 4%/12 in those missing months                        
                        
ReturnS = sorted(Re)    # Sort the returns in increasing order

T1 = .05 * T             #  5% of the sample
T1 = round(T1)           # Round the number to an integer

Value1 = 1

for i in range(T-T1):
    Value1 = Value1 * (1 + ReturnS[i])

for i in range(T - T1 + 1, T):
    Value1 = Value1 * (1 + 0.04/12)

print('Q2: The accumulative return if we miss 5% of the best months? \n')
print('          {0:.2f}\n'.format(Value1))

In [ ]:
# Q3: What is the max drawdown, the largest % drop from a previous peak

       #Note:  We just copy and paste the algorithm (see the zip file) and take its validity
           # for granted, as in most cases in practice for well established theories or formulas.
       # Since the algorithm requires prices as input, we translate the
       # returns into prices first (the starting price can be set as 100)
 
P = 100*np.ones((T,1))     # initial values

MDD = 0
Worst = 0
             # using the algorithm in the file
max = -99
Worst = np.amin(Re)         # the minimum of an array, the return here

Ptemp = 100                # need a temp to execute P[j+1]=P[j]*(1+Re[j]), so that
                             # P[j+1] is stored as P[j], to avoid index out of range in Python

for j in range(T): 
    Ptemp=Ptemp*(1+Re[j])        # convert to prices;
    P[j]=Ptemp   
    if P[j]>max:
        max=P[j]
    DD=100*(max-P[j]) / max
    if DD>MDD:
        MDD=DD
        
MDD = MDD / 100             # in percentage 
print(MDD)

In [ ]:
# Simulate T normal random varables with the same mean and varance  

e = np.random.randn(T,1)     # Generate data from N(0,1), T by 1  

RN = np.ones((T,1))          # create the storage

for i in range(T):           # transform the data so that its mean and variance match the mkt
     RN[i]=mu + sigma*e[i]          # RN = mu*np.ones((T,1)) + sigma*e does the same, but 
                                    # month by month may be easier to understand
 


In [ ]:
# plot the returns and simulated returns

p=plt.plot(Re)
p1 = plt.plot(RN)

In [ ]:
# plot the histogram of returns and simulated returns

p=plt.hist(Re)
p1 = plt.hist(RN)

In [ ]:
# plot the index and its 12-month moving-average  
 
df['12MA'] = df['Index'].rolling(window=12,min_periods=0).mean()
                # this will create a column, named as 12MA, from the index by taking the average
                       # of the past 12 month data (including current month)
                # At the firtst to 11 month, impossible to do it. min_periods=0 indicate repplaing them
                   # by using the average of available original data
                
print(df.head())
print(df.tail())

p2= plt.plot(df['Index'])
p3 = plt.plot(df['12MA'])

In [ ]:
# plot a subperiod
 

p2= plt.plot(df.loc[0:100,'Index'], label='SP500 Index')
p3 = plt.plot(df.loc[0:100,'12MA'], label='12 month MA')

plt.xlabel('Month')
plt.ylabel('Index Level')
plt.title('SP500 and \n Its 12-month MA')
plt.legend()
plt.show()
